In [ ]:
%load_ext autoreload
%autoreload 2

import os
import glob
from yaml import safe_load as yaml_load

import numpy as np
import tifffile
import matplotlib
import matplotlib.pyplot as plt

from stardist import random_label_cmap

from keras_transfer_learning import model, dataset
from keras_transfer_learning.heads import segm

lbl_cmap = random_label_cmap()

In [ ]:
config = {
    'name': 'test_segm_weighted',
    'input_shape': [128, 64, 1],
    'data': {
        'name': 'stardist-dsb2018',
        'data_dir': 'data/stardist-dsb2018',
        'train_val_split': 0.9,
        'normalizer': 'min-max',
        'datasplit_seed': 42,
    },
    'backbone': {
        'name': 'unet',
        'args': {  
            'filters': [32, 64, 128]
        },
        'weights': None
    },
    'head': {
        'name': 'fgbg-segm-weighted',
        'args': {},
        'prepare_model_args': {},
        'prepare_data_args': {}
    }
}

In [ ]:
seed = None # Change to load another example

d = dataset.Dataset(config)

img, mask = d.get_random_test_img()

plt.subplot(1, 2, 1)
plt.imshow(img)
plt.subplot(1, 2, 2)
plt.imshow(mask, cmap=lbl_cmap)
plt.show()

In [ ]:
batch_x = [img]
batch_y = [mask]

prepare_data = segm.prepare_data_fgbg_weigthed
(out_x, weight_map), out_y = prepare_data(batch_x, batch_y, border_weight=6, separation_border_weight=20)

print(out_x.shape, weight_map.shape, out_y.shape)
print(np.min(weight_map), np.max(weight_map))

plt.figure(figsize=(9, 4))
plt.subplot(1, 3, 1)
plt.imshow(out_x[0,...,0])
plt.subplot(1, 3, 2)
plt.imshow(weight_map[0,...])
plt.subplot(1, 3, 3)
plt.imshow(out_y[0,...,0])
plt.show()